In [ ]:
# Import libraries

import os
import time
import warnings
import numpy as np
from scipy import stats
import matplotlib as mpl
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

# Disble ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
# Global variables

# Number of samples
num_bins =10
num_samples_per_bin = 10
num_samples = 2*num_bins*num_samples_per_bin

# Number of shots
num_shots = 1024

# SA parameters
num_runs = 20
num_sa_iters = 50
ini_temp = 10
cool_coef = 0.95

# Create directory
today = date.today()
today = str(today)
today = "%s-%s-%s"%(today.split('-')[0],today.split('-')[1],today.split('-')[2])

date_dir = '../data/2020-05-23'
if not os.path.exists(date_dir):
    os.makedirs(date_dir)

In [ ]:
train_o_p = np.load(date_dir + '/train_o_p.npy')
train_d_1 = np.load(date_dir + '/train_d_1.npy')

In [ ]:
# Discriminant number of parameters and bounds
# For Ell2D: origin-x, origin-y, radius-x, radius-y, angle of rotation X 2
num_params = 10
bounds = [[-8, 0], [3, 10], [0, 2*np.pi], [5, 10], [5, 10], [2, 10], [-10, -5], [0, 2*np.pi], [5, 10], [5, 10]]

# Sample the performance of a configuration
def sample_perf(config, obj):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    
    A0 = (np.cos(config[2])/config[3])**2 + (np.sin(config[2])/config[4])**2
    B0 = 2 * np.cos(config[2]) * np.sin(config[2]) * ((1/config[3]**2) - (1/config[4]**2))
    C0 = (np.sin(config[2])/config[3])**2 + (np.cos(config[2])/config[4])**2
    
    A1 = (np.cos(config[7])/config[8])**2 + (np.sin(config[7])/config[9])**2
    B1 = 2 * np.cos(config[7]) * np.sin(config[7]) * ((1/config[8]**2) - (1/config[9]**2))
    C1 = (np.sin(config[7])/config[8])**2 + (np.cos(config[7])/config[9])**2
        
    for i in range(len(train_d_1)):
        x0 = np.real(train_d_1[i]) - config[0]
        y0 = np.imag(train_d_1[i]) - config[1]
        frac_0 = sum(A0*x0**2 + B0*x0*y0 + C0*y0**2 <= 1)
        x1 = np.real(train_d_1[i]) - config[5]
        y1 = np.imag(train_d_1[i]) - config[6]
        frac_1 = sum(A1*x1**2 + B1*x1*y1 + C1*y1**2 <= 1)
        if frac_0 + frac_1 != 0:
            frac_0 /= (frac_0 + frac_1)
            o_pf_up[i] = abs(train_o_p[i]-frac_0)
            o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = None
    perf_dn = None
    if obj == 'median':
        perf_up = np.median(o_pf_up)
        perf_dn = np.median(o_pf_dn)
    elif obj == 'spread':
        perf_up = np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25)
        perf_dn = np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25)
    elif obj == 'mdnspd':
        perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
        perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine(obj):
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config, obj)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config, obj)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list

for obj in ['median', 'spread', 'mdnspd']:
    times = []
    confs = []
    perfs = []
    for k in range(num_runs):
        start = time.time()
        config_list, perf_list = simulated_annealing_engine(obj)
        end = time.time()

        config = config_list[perf_list.index(min(perf_list))]
        perf = min(perf_list)

        print(obj, k, end-start, config, perf)

        times.append(end-start)
        confs.append(config)
        perfs.append(perf)

    np.save(date_dir + '/E2D_' + obj + '_runtime.npy', times)
    np.save(date_dir + '/E2D_' + obj + '_configs.npy', confs)
    np.save(date_dir + '/E2D_' + obj + '_perform.npy', perfs)

In [ ]:
# Discriminant number of parameters and bounds
# For Cir2D: origin-x, origin-y, radius X 2
num_params = 6
bounds = [[-8, 0], [3, 10], [5, 10], [2, 10], [-10, -5], [5, 10]]

# Sample the performance of a configuration
def sample_perf(config, obj):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = sum(np.absolute(train_d_1[i] - complex(config[0], config[1])) <= config[2])
        totals = frac_0 + sum(np.absolute(train_d_1[i] - complex(config[3], config[4])) <= config[5])
        if totals != 0:
            frac_0 /= totals
            o_pf_up[i] = abs(train_o_p[i]-frac_0)
            o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = None
    perf_dn = None
    if obj == 'median':
        perf_up = np.median(o_pf_up)
        perf_dn = np.median(o_pf_dn)
    elif obj == 'spread':
        perf_up = np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25)
        perf_dn = np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25)
    elif obj == 'mdnspd':
        perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
        perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine(obj):
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config, obj)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config, obj)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list

for obj in ['median', 'spread', 'mdnspd']:
    times = []
    confs = []
    perfs = []
    for k in range(num_runs):
        start = time.time()
        config_list, perf_list = simulated_annealing_engine(obj)
        end = time.time()

        config = config_list[perf_list.index(min(perf_list))]
        perf = min(perf_list)

        print(obj, k, end-start, config, perf)

        times.append(end-start)
        confs.append(config)
        perfs.append(perf)

    np.save(date_dir + '/C2D_' + obj + '_runtime.npy', times)
    np.save(date_dir + '/C2D_' + obj + '_configs.npy', confs)
    np.save(date_dir + '/C2D_' + obj + '_perform.npy', perfs)

In [ ]:
'''
# Discriminant number of parameters and bounds
# For LinD: origin-x, origin-y, radius
num_params = 2
bounds = [[0, 5], [-25, 0]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = 0
        for s in train_d_1[i]:
            y = config[0]*np.real(s) + config[1]
            if np.imag(s) >= y:
                frac_0 += 1
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/LinD_runtimes.npy', times)
np.save(date_dir + '/LinD_configurations.npy', confs)
np.save(date_dir + '/LinD_performances.npy', perfs)
'''

In [ ]:
'''
# Discriminant number of parameters and bounds
# For QuaD: origin-x, origin-y, radius
num_params = 4
bounds = [[0, 20], [-10, 10], [0, 2*np.pi], [0.1, 5]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = 0
        for s in train_d_1[i]:
            x = np.real(s) - config[0]
            y = np.imag(s) - config[1]
            r = np.sqrt(x**2 + y**2)
            t = np.arctan2(y, x) - config[2]
            x = (r * np.cos(t)) + config[0]
            y = (r * np.sin(t)) + config[1]
            if y >= (config[3]*((x-config[0])**2)+config[1]):
                frac_0 += 1
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/QuaD_runtimes.npy', times)
np.save(date_dir + '/QuaD_configurations.npy', confs)
np.save(date_dir + '/QuaD_performances.npy', perfs)
'''

In [ ]:
'''
# Discriminant number of parameters and bounds
# For CirD: origin-x, origin-y, radius
num_params = 3
bounds = [[-8, 10], [-10, 10], [5, 10]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = sum(np.absolute(train_d_1[i] - complex(config[0], config[1])) <= config[2])
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/CirD_runtimes.npy', times)
np.save(date_dir + '/CirD_configurations.npy', confs)
np.save(date_dir + '/CirD_performances.npy', perfs)
'''